In [ ]:
import numpy as np
import pandas as pd
import time
import torch as T
device = T.device("cuda")

In [ ]:
class Feature_dataset(T.utils.data.Dataset):

  def __init__(self,features,label,m_rows=None):
    self.f_data = T.tensor(features,dtype = T.float32).to(device)
    self.y_data = T.tensor(label,dtype = T.float32).to(device)

  def __len__(self):
    return (len(self.f_data))

  def __getitem__(self,idx):
    preds = self.f_data[idx,:]
    score = self.y_data[idx,:]
    return (preds, score)



In [ ]:
class network(T.nn.Module):
  def __init__(self):
    super(network,self).__init__()
    self.hid1 = T.nn.Linear(3,300)
    #self.dropout1 = T.nn.Dropout(0.2)
    self.hid2 = T.nn.Linear(300,200)
    #self.dropout2 = T.nn.Dropout(0.4)
    self.oupt = T.nn.Linear(200,1)

    T.nn.init.xavier_uniform_(self.hid1.weight)
    T.nn.init.zeros_(self.hid1.bias)
    T.nn.init.xavier_uniform_(self.hid2.weight)
    T.nn.init.zeros_(self.hid2.bias)
    T.nn.init.xavier_uniform_(self.oupt.weight)
    T.nn.init.zeros_(self.oupt.bias)

  def forward(self,x):
    z = T.relu(self.hid1(x))
    #z = self.dropout1(z)
    #print(z)
    z = T.relu(self.hid2(z))
    #z = self.dropout2(z)
    z = self.oupt(z)

    return z

In [ ]:
def accuracy(model,xdat,pct):
  n = len(xdat)
  X = xdat[0:n][0]  # all predictor values
  Y = xdat[0:n][1]
 

  with T.no_grad():
    oupt = model(X)

  tol_Delta = T.abs(pct*Y)
  abs_Delta = T.abs(oupt - Y)
  print(oupt)

  results = abs_Delta < tol_Delta
  acc = T.sum(results,dim=0).item()/n
  return acc

In [ ]:
# Training and Testing
from sklearn.model_selection import train_test_split
T.manual_seed(4)
np.random.seed(4)


train_f = pd.read_csv("actual_score_data.csv")

n = len(train_f)
ftr_score = train_f.iloc[:,3:6].to_numpy()
scores = train_f.iloc[:,6:7].to_numpy().reshape(-1,1)
X_train,X_test,y_train,y_test = train_test_split(ftr_score,scores,test_size=0.2,random_state=42)

train_dat = Feature_dataset(X_train,y_train)
test_dat = Feature_dataset(X_test,y_test)
bat_size = 50
train_ldr = T.utils.data.DataLoader(train_dat,batch_size= bat_size,shuffle = True)

# Creating Network
net = network().to(device)

max_epochs = 500
ep_log_interval = 5
lrn_rate = 0.09

loss_func = T.nn.MSELoss()

optimizer = T.optim.Adam(net.parameters(),lr = lrn_rate)

print("\nbat_size = %3d " % bat_size)
print("\nbat_size = %3d " % bat_size)
print("loss = " + str(loss_func))
print("optimizer = Adam")
print("max_epochs = %3d " % max_epochs)
print("lrn_rate = %0.3f " % lrn_rate)
print("\nStarting training with saved checkpoints")





bat_size =  50 

bat_size =  50 
loss = MSELoss()
optimizer = Adam
max_epochs = 500 
lrn_rate = 0.090 

Starting training with saved checkpoints


In [ ]:
net.train()

for epoch in range(0,max_epochs):
  T.manual_seed(1+epoch)
  epoch_loss=0

  for(batch_idx,batch) in enumerate(train_ldr):
    (X,Y) = batch
    optimizer.zero_grad()
    oupt = net(X)
    #print(oupt)
    #print("Actual: {}  Output: {}".format(Y,oupt))
    loss_val = loss_func(oupt,Y)
    epoch_loss += loss_val.item()
    loss_val.backward()
    optimizer.step()

  if epoch % ep_log_interval ==0:
    print("epoch = %4d   loss = %0.4f" % \
       (epoch, epoch_loss))
    dt = time.strftime("%Y_%m_%d-%H_%M_%S")
    fn = ".\\Log\\" + str(dt) + str("-") + \
       str(epoch) + "_checkpoint.pt"

    info_dict = {
        'epoch' : epoch,
        'net_state': net.state_dict(),
        'optimizer_state': optimizer.state_dict()
    }
    T.save(info_dict, fn)

print("Done ")


epoch =    0   loss = 1206.9705
epoch =    5   loss = 148.1736
epoch =   10   loss = 202.1978
epoch =   15   loss = 202.0688
epoch =   20   loss = 202.2444
epoch =   25   loss = 201.7909
epoch =   30   loss = 202.4001
epoch =   35   loss = 202.3216
epoch =   40   loss = 202.4831
epoch =   45   loss = 202.3447
epoch =   50   loss = 202.1081
epoch =   55   loss = 202.1813
epoch =   60   loss = 201.6853
epoch =   65   loss = 202.5164
epoch =   70   loss = 202.1113
epoch =   75   loss = 202.0444
epoch =   80   loss = 202.4517
epoch =   85   loss = 202.6427
epoch =   90   loss = 201.6701
epoch =   95   loss = 201.3047
epoch =  100   loss = 202.1104
epoch =  105   loss = 202.0920
epoch =  110   loss = 202.0813
epoch =  115   loss = 202.1760
epoch =  120   loss = 202.3025
epoch =  125   loss = 202.2209
epoch =  130   loss = 202.3855
epoch =  135   loss = 202.7888
epoch =  140   loss = 202.0074
epoch =  145   loss = 201.5980
epoch =  150   loss = 202.1804
epoch =  155   loss = 201.6958
epoch =

In [ ]:
print(net(test_dat[9][0]))

tensor([0.8648], device='cuda:0', grad_fn=<AddBackward0>)


In [ ]:
print("\nComputing model accuracy")
net.eval()
acc_train = accuracy(net,train_dat, 0.40) 
print("Accuracy (within 0.20) on train data = %0.4f" % \
      acc_train)

acc_test = accuracy(net,test_dat, 0.20) 
print("Accuracy (within 0.20) on test data  = %0.4f" % \
      acc_test)


Computing model accuracy
tensor([[0.2571],
        [0.2571],
        [0.2571],
        ...,
        [0.2571],
        [0.2571],
        [0.2571]], device='cuda:0')
Accuracy (within 0.20) on train data = 0.0558
tensor([[0.2571],
        [0.2571],
        [0.2571],
        ...,
        [0.2571],
        [0.2571],
        [0.2571]], device='cuda:0')
Accuracy (within 0.20) on test data  = 0.0132


In [ ]:
import pandas as pd
df = pd.read_csv("Analysis_Trial_final_2.csv")



In [ ]:

lst2 = [i for i in lst if str(i)!='nan']
len(lst2)

99992

In [ ]:
df["Actual"] = lst2

In [ ]:
df.to_csv("actual_score_data.csv")

In [ ]:
df.iloc[:,6:7]

,Actual
0,1.319510
1,1.325713
2,1.308239
3,0.458899
4,0.929964
...,...
99987,1.301711
99988,-0.882188
99989,1.162358
99990,0.728620
